In [1]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem
import pandas as pd
import rdchiral
import itertools
import pickle
import sys
from tqdm import tqdm 
from rdchiral.main import rdchiralRun, rdchiralRunText, rdchiralReaction, rdchiralReactants
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
import matplotlib.pyplot as plt
import multiprocessing
from tqdm import tqdm
sys.path.append('../../retrosynthesis/')

from common import *

### input and output

In [2]:
# step1
#input
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter.csv'
yeast870_path = '../../../Data/model/yeast-GEM-final.csv'
# new_met_info_to_GEM_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_top50_0.3.csv'#######换成筛选后的
new_met_info_to_GEM_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'
target_100met_path = '../../../Data/100met_with_smiles.xlsx'

example_met = 'Psilocybin'

In [3]:
def process_rule(rule_smiles, mets_total_smiles):
    score = []
    smiles = []
    if 'C' in rule_smiles or 'c' in rule_smiles:
        for j in mets_total_smiles:
            current_score = calculate_similarity(rule_smiles, j)
            if current_score > 0.3:
                score.append(current_score)
                smiles.append(j)
    else:
        smiles.append(rule_smiles)
        score.append(1)

    if len(score) == 0:
        return rule_smiles, [], []
    elif len(score) > 20:
        top40_data = sorted(zip(score, smiles), reverse=True)[:20]
    else:
        top40_data = sorted(zip(score, smiles), reverse=True)
    
    top40_scores, top40_smiles = zip(*top40_data)
    
    return rule_smiles, list(top40_smiles), list(top40_scores)

In [4]:
def process_retrorule(index_row, mets_total_smiles,retrosys_smiles_calculate_similarity_pd,heterologous_met_smiles):
    rxndb_list_tmp = []
    # newdbSmiles_tmp = []
    failedrxn_tmp = []
    index, row = index_row
    rxn_id, ECnumber, reactants, products = row['MNX_ID'], row['classifs'], row['substrate_smiles'], row['product_smiles']
    deprecated_equ_smiles, reactantSMARTs, productSMARTs, rule = row['deprecated_equ_smiles'], row['ReactantsSMARTs'], row['ProductSMARTs'], row['RetroRules']
    deprecated_equ_smiles = deprecated_equ_smiles.split('>>')
    if rxn_id not in ['MNXR101882', 'MNXR101884', 'MNXR101879', 'MNXR101885', 'MNXR101886', 'MNXR101887',
                        'MNXR103528', 'MNXR103575']:  # ATP NADH conversion stuff
        productSMARTs = productSMARTs.split('.')
        products = products.split('.')
        new_products_smiles = []
        scores = []

        if len(productSMARTs) == len(products):
            for x in range(len(productSMARTs)):  # prepare mets with substructure and sort the reactants combination
                new_products_smile_one,score_one = get_calculate_similarity(products[x],retrosys_smiles_calculate_similarity_pd)
                new_products_smiles.append(new_products_smile_one)
                scores.append(score_one)

            product_prepared = []
            scores_prepared = []
            for lst, score_lst in zip(new_products_smiles, scores):
                if heterologous_met_smiles in lst:
                    if len(new_products_smiles) == 1:  # if only one sub list
                        product_prepared.append([heterologous_met_smiles])
                        scores_prepared.append(score_lst)
                    else:
                        other_lists = [sub_lst for sub_lst in new_products_smiles if sub_lst != lst]
                        tmp_product = list(itertools.product([heterologous_met_smiles], *other_lists))
                        product_prepared += tmp_product

                        tmp_scores = list(itertools.product([score_lst[new_products_smiles.index(lst)]], *scores))
                        scores_prepared += tmp_scores

            for m in range(0, len(product_prepared)):  # apply the rule and output the rxn
                smile = product_prepared[m]
                score = scores_prepared[m]
                
                if len(smile) > 1:
                    product_smile = '.'.join(smile)
                else:
                    product_smile = smile[0]

                retrorule_tmp = rule.split('>>')
                retrorule_tmp = '(' + retrorule_tmp[0] + ')>>' + '(' + retrorule_tmp[1] + ')'            
                product_smile_tmp = product_smile
                try:
                    # print('product_smile_tmp',product_smile_tmp)
                    product_mol = Chem.MolFromSmiles(product_smile_tmp)
                    reaction = AllChem.ReactionFromSmarts(retrorule_tmp)
                    reactant_smiles = reaction.RunReactants((product_mol,))
                    smiles = [Chem.MolToSmiles(x[0]) for x in reactant_smiles]
                    reactant_smiles = list(set(smiles))

                    for reactant_smile in reactant_smiles:
                        if len(deprecated_equ_smiles) > 0:
                            if deprecated_equ_smiles[0] == '':
                                reactant_smile_final = reactant_smile
                            else:
                                reactant_smile_final = reactant_smile + '.' + deprecated_equ_smiles[0]
                            if deprecated_equ_smiles[1] == '':
                                product_smile_final = product_smile_tmp
                            else:
                                product_smile_final = product_smile_tmp + '.' + deprecated_equ_smiles[1]
                        rxn_smiles = reactant_smile + '>>' + product_smile_tmp
                        rxn_smiles_final = reactant_smile_final + '>>' + product_smile_final
                        
                        tmp = {
                            'EC number': ECnumber,
                            'rule': rule,
                            'templateID': rxn_id,
                            'templateSubstrate': products,
                            'rxn_smiles_basic': rxn_smiles,
                            'rxn_smiles_final': rxn_smiles_final,
                            'reactant_smile': reactant_smile,
                            'productsmile': product_smile_tmp,
                            'similarity': score
                        }
                        if check_rule(tmp['rxn_smiles_basic'])=='blance':
                            rxndb_list_tmp.append(tmp)  
                            # mets_tmp = reactant_smile.split('.')      
                            # for reactant in mets_tmp:             
                            #     if reactant not in mets_total_smiles and reactant not in newdbSmiles_tmp:         
                            #         newdbSmiles_tmp.append(reactant)        
                except:
                    failedrxn_tmp.append(rxn_id)
                
    return(rxndb_list_tmp,failedrxn_tmp)

In [5]:
rules_product_smiles_lst = []
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
for index,row in retrorules.iterrows():
    tmp  = row['product_smiles'].split('.')
    for i in tmp:
        if i not in rules_product_smiles_lst:
            rules_product_smiles_lst.append(i)
print('rules_product_smiles_lst',len(rules_product_smiles_lst))

rules_product_smiles_lst 9708


In [6]:
target_100met = pd.read_excel(target_100met_path)
target_100met = target_100met[target_100met['Group']=='heterologous']
target_100met = target_100met[~target_100met['Smiles'].isna()]
target_100met.reset_index(drop=True,inplace=True)
target_100met['byproduct'] = ' '
target_100met['EC number'] = ' '
target_100met.head()

,Name,Formula,KEGG ID,CHEBI ID,Group,class,Gene target,Smiles,byproduct,EC number
0,L-Phenylacetylcarbinol,C9H10O2,NaN,NaN,heterologous,alcohol,Link,CC(=O)[C@H](C1=CC=CC=C1)O,,
1,1-Butanol,C4H10O,C06142,28885.0,heterologous,alcohol,Link,CCCCO,,
2,(S)-Reticuline,C19H23NO4,C02105,16718.0,heterologous,alkaloid,Link,CN1CCC2=CC(=C(C=C2[C@@H]1CC3=CC(=C(C=C3)OC)O)O)OC,,
3,Psilocybin,C12H17N2O4P,C07576,8614.0,heterologous,alkaloid,Link,CN(C)CCC1=CNC2=C1C(=CC=C2)OP(=O)(O)O,,
4,Nicotianamine,C12H21N3O6,C05324,17721.0,heterologous,amino acid,Link,C1CN([C@@H]1C(=O)O)CC[C@@H](C(=O)O)NCC[C@@H](C...,,


In [7]:
len(target_100met)

44

## byproduct prediction

In [8]:
def check_productsmile(smiles):#only keep smiles with carbon
    smiles_lst = smiles.split('.')
    smiles_lst = [x for x in smiles_lst if smiles_has_carbon(x)]
    return len(smiles_lst)

In [9]:
target_100met

,Name,Formula,KEGG ID,CHEBI ID,Group,class,Gene target,Smiles,byproduct,EC number
0,L-Phenylacetylcarbinol,C9H10O2,NaN,NaN,heterologous,alcohol,Link,CC(=O)[C@H](C1=CC=CC=C1)O,,
1,1-Butanol,C4H10O,C06142,28885.0,heterologous,alcohol,Link,CCCCO,,
2,(S)-Reticuline,C19H23NO4,C02105,16718.0,heterologous,alkaloid,Link,CN1CCC2=CC(=C(C=C2[C@@H]1CC3=CC(=C(C=C3)OC)O)O)OC,,
3,Psilocybin,C12H17N2O4P,C07576,8614.0,heterologous,alkaloid,Link,CN(C)CCC1=CNC2=C1C(=CC=C2)OP(=O)(O)O,,
4,Nicotianamine,C12H21N3O6,C05324,17721.0,heterologous,amino acid,Link,C1CN([C@@H]1C(=O)O)CC[C@@H](C(=O)O)NCC[C@@H](C...,,
5,4-Hydroxymandelic,C8H8O4,C11527,16388.0,heterologous,aromatic,Link,C1=CC(=CC=C1C(C(=O)O)O)O,,
6,Cinnamate,C9H8O2,C00423,15669.0,heterologous,aromatic,Link,C1=CC=C(C=C1)/C=C/C(=O)[O-],,
7,Mandelic,C8H8O3,C01984,17756.0,heterologous,aromatic,Link,C1=CC=C(C=C1)[C@@H](C(=O)O)O,,
8,p-Coumaric acid,C9H8O3,C00811,32374.0,heterologous,aromatic,Link,C1=CC(=CC=C1/C=C/C(=O)O)O,,
9,Rosmarinate,C18H16O8,C01850,50371.0,heterologous,aromatic,Link,C1=CC(=C(C=C1C[C@H](C(=O)O)OC(=O)/C=C/C2=CC(=C...,,


In [10]:
for index,row in tqdm(target_100met.iterrows(),total=len(target_100met)):
    if row['Name'] == example_met :
        # if row['byproduct']==' ':
        print(row['Name'])
        # heterologous_met_smiles = row['Smiles']
        heterologous_met_smiles = 'C1=CC=C(C=C1)C=O'
        heterologous_met_smiles = normalize_smiles(heterologous_met_smiles)

        model_met = get_GEM_all_smiles(yeast870_path)
        new_met_info_to_GEM = pd.read_csv(new_met_info_to_GEM_path,index_col=None)
        new_met_info_to_GEM.head(6)
        new_met = new_met_info_to_GEM[new_met_info_to_GEM['ID'].apply(lambda x:'sn_' in x)]['new_met_smiles'].to_list()
        new_met = [normalize_smiles(x) for x in new_met if 'C' in x or 'c' in x]
        # new_met = []
        # add heterologous_met_smiles to model_met list
        new_model_and_heterologous_met = model_met + new_met
        new_model_and_heterologous_met.append(heterologous_met_smiles)
        new_model_and_heterologous_met = [smile for smile in new_model_and_heterologous_met if '.' not in smile]
        new_model_and_heterologous_met = [smile for smile in new_model_and_heterologous_met if 'C' in smile or 'c' in smile] ###只筛第一轮
        print(len(new_model_and_heterologous_met))
        #calculate similarity
        num_processes = 60
        pool = multiprocessing.Pool(num_processes)
        retrosys_smiles_calculate_similarity = {'rules_smiles': [],
                                                'smiles_in_mets_total_smiles': [],
                                                'scores': []}


        process_rule_partial = partial(process_rule, mets_total_smiles=new_model_and_heterologous_met)
        for result in tqdm(pool.imap(process_rule_partial, rules_product_smiles_lst, chunksize=50), total=len(rules_product_smiles_lst)):
            if result[0] not in retrosys_smiles_calculate_similarity['rules_smiles']:
                retrosys_smiles_calculate_similarity['rules_smiles'].append(result[0]) 
                retrosys_smiles_calculate_similarity['smiles_in_mets_total_smiles'].append(result[1]) 
                retrosys_smiles_calculate_similarity['scores'].append(result[2]) 
        pool.close()
        pool.join()

        retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)

        rxndb_list = []
        # newdbSmiles = []
        failedrxn = []
        num_processes =60  #  adjust the number of processes according to your needs
        pool = multiprocessing.Pool(num_processes)
        # Execute in parallel using process pool

        # Create a partial function that passes mets_total_smiles and mets_total_mol as default parameters to the process_retrorule function
        process_retrorule_partial = partial(process_retrorule, mets_total_smiles=new_model_and_heterologous_met,retrosys_smiles_calculate_similarity_pd = retrosys_smiles_calculate_similarity_pd,heterologous_met_smiles = heterologous_met_smiles)

        # Using the process pool to execute in parallel, the chunksize parameter is placed inside the imap function.
        for result in tqdm(pool.imap(process_retrorule_partial, retrorules.iterrows(), chunksize=50), total=len(retrorules)):
            # if result == ([], [], []):
            #     pass
            # else:
            #     print(result)
            if result[0] :  #  check if the result is not empty
                rxndb_list.extend(result[0]) 
            # if result[1] is not None:
            #     newdbSmiles.extend(result[1])
            if result[1]:
                failedrxn.extend(result[1])  

        pool.close()
        pool.join()

        rxndb = {}
        for i, rxn_dict in enumerate(rxndb_list):
            # generate a unique key for each reaction, for example "rxn0"，"rxn1"，"rxn2"，...
            key = f'rxn{i+1}'
            # merge to the dictionary
            rxndb[key] = rxn_dict
        # with open(rxndb_path, 'w') as json_file:
        #     json.dump(rxndb, json_file)

        # with open(rxndb_path, 'r') as f:
        #     rxndb = json.load(f)
        print('rxndb num',len(rxndb))
        # print('newdbSmiles num',len(newdbSmiles))
        print('failedrxn num',len(failedrxn))

        miss_smiles = ['C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C(N=CN=C54)N)O)O)O)O',#NADH
                        'C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C(N=CN=C54)N)OP(=O)(O)O)O)O)O']#NADPH
        miss_reaction_lst = []
        for k,v in rxndb.items():
            reactant_smile_lst = v['rxn_smiles_final'].split('>>')[0].split('.')
            if any(compare_smiles_inchikey(miss_smiles[0],met) for met in reactant_smile_lst):
                # print(k,v['rxn_smiles_final'])
                miss_reaction_lst.append(k)
            if any(compare_smiles_inchikey(miss_smiles[1],met) for met in reactant_smile_lst):
                # print(k,v['rxn_smiles_final'])    
                miss_reaction_lst.append(k)
        print(len(miss_reaction_lst))
        
        result_df = {'ID':[],
                    'reactant_smile':[],
                    'productsmile':[]}
        for k,v in rxndb.items():
            result_df['ID'].append(k)
            result_df['reactant_smile'].append(v['reactant_smile'])
            result_df['productsmile'].append(v['productsmile'])
        result_df = pd.DataFrame({'ID': result_df['ID'],
                                'reactant_smile': result_df['reactant_smile'],
                                'productsmile': result_df['productsmile']})
        result_df.drop_duplicates(subset=['reactant_smile', 'productsmile'], inplace=True)
        result_df.reset_index(drop=True, inplace=True)

        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.OO']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=C=O']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=S([O-])[O-]']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.[SH-]']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=P([O-])([O-])OP(=O)([O-])O']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.II']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.[NH4+]']
        result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.[Br-]']

        unique_smiles = []
        for i in result_df['reactant_smile']:
            if all(compare_smiles_inchikey(x,i)!=1 for x in unique_smiles):
                unique_smiles.append(i)
            else:
                pass
        result_df = result_df[result_df['reactant_smile'].isin(unique_smiles)]

        unique_productsmile = []
        for i in result_df['productsmile']:
            if i.replace('O=P(O)(O)','') not in unique_productsmile:
                unique_productsmile.append(i)
            else:
                pass
        result_df = result_df[result_df['productsmile'].isin(unique_productsmile)]

        result_df['productsmile_num'] = result_df['productsmile'].apply(lambda x: check_productsmile(x))
        result_df = result_df[result_df['productsmile_num']==1]
        result_df = result_df[~result_df['ID'].isin(miss_reaction_lst)]

        res = result_df['reactant_smile'].to_list()
        ec_lst = []
        for k,v in rxndb.items():
            if v['reactant_smile'] in res:
                ec_lst.append(v['EC number'])

        print(row['Name'],len(result_df))
        print(row['Name'],ec_lst)

        # target_100met.loc[index,'byproduct'] = str(res)
        target_100met.loc[index,'byproduct'] = len(result_df)
        target_100met.loc[index,'EC number'] = str(ec_lst)
        # target_100met.to_csv(heterologous_met_byproduct_result_path,index=None)

  0%|          | 0/44 [00:00<?, ?it/s]

Psilocybin
1884


100%|██████████| 21921/21921 [00:20<00:00, 1071.68it/s]


rxndb num 125
failedrxn num 0
17


100%|██████████| 44/44 [11:41<00:00, 15.95s/it]  

Psilocybin 12
Psilocybin ['1.1.1;1.1.1.19;1.1.1.2;1.1.1.21;1.1.1.91', '1.17.3.2;1.2.3.1;1.2.3.14;1.2.3.9', '1.14.12.12', '1.14.12.12', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.1.1.90;1.1.1.M33', '1.1.1.90', '1.14.13.-', '1.14.13.-', '1.14.13.-', '1.13.11.-', '1.13.11.-', '1.1.1.1', '1.1.1.1', '1.14.14.-;1.14.14.1', '1.14.14.-;1.14.14.1', '1.14.12.-', '1.14.12.-', '1.14.13.244', '1.14.13.244', '1.14.13.244', '1.1.1.21;1.1.1.90', '1.4.3.13;1.4.3.21;1.4.3.4;1.5.3.17', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21;1.1.1.51', '1.1.1;1.1.1.21', '1.1.1.21', '1.1.1.2;1.1.1.21', '1.1.1.19;1.1.1.2;1.1.1.21;1.1.1.213;1.1.1.51;1.1.1.62', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1.21', '1.1.1;1.1.1.184;1.1.1.2;1.1.1.21;1.1.1.50;1.1.1.65', '1.11;1.11.2.1', '1.11;1.11.2.1', '1.11;1.11.2.1', '1.14.13.84', '1.11.2.1', '1.1.1.2;1.1.1.21', '1.1.1.2;1.1.1.21', '1.1.1.184;1.1.1.2', '1.14.13.-;1.14.13.243', '1.14.13.-;1.14.13.243', '1.

In [11]:
result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.[Br-]']
result_df = result_df[result_df['productsmile']!=heterologous_met_smiles+'.O=S(=O)([O-])[O-]']

In [12]:
import pubchempy as pcp
import time

def get_cid_from_smiles(smiles):
    try:
        # Search for the compound using its SMILES
        compound = pcp.get_compounds(smiles, 'smiles') or pcp.get_compounds(neutralize_charge(smiles), 'smiles')
        time.sleep(1)
        if compound:
            return compound[0].cid
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

result_df[result_df['reactant_smile'].apply(lambda x:all(get_cid_from_smiles(smiles) for smiles in x.split('.')))]

,ID,reactant_smile,productsmile,productsmile_num
1,rxn2,OCc1ccccc1,O=Cc1ccccc1,1
2,rxn3,O=C([O-])c1ccccc1.OO,O=Cc1ccccc1.O=O.[H]O[H],1
7,rxn9,O=Cc1cC=C[C@H](O)[C@@H]1O,O=Cc1ccccc1.O=O,1
8,rxn10,O.O=Cc1ccc(O)cc1,O=Cc1ccccc1.O=O,1
9,rxn11,O.O=Cc1ccccc1O,O=Cc1ccccc1.O=O,1
10,rxn12,O.O=Cc1cccc(O)c1,O=Cc1ccccc1.O=O,1
33,rxn62,O.O=O.[NH3+]Cc1ccccc1,O=Cc1ccccc1.[NH4+].OO,1
37,rxn91,O.O=C([O-])c1ccccc1,O=Cc1ccccc1.O=O,1
38,rxn92,OCc1ccccc1.OO,O=Cc1ccccc1.[H]O[H].[H]O[H],1


In [13]:
# ground truth BZOA-benzoic acid(C1=CC=C(C=C1)C(=O)O)  BZOH-benzyl alcohol(C1=CC=C(C=C1)CO)

print('====================')
true_met = 'C1=CC=C(C=C1)C(=O)O'
for index,row in result_df.iterrows():
    reactant_smile_lst = row['reactant_smile'].split('.')
    if any(compare_smiles_inchikey(true_met,met) for met in reactant_smile_lst):
        print(row['ID'],row['reactant_smile'])

print('====================')
true_met = 'C1=CC=C(C=C1)CO'
for index,row in result_df.iterrows():
    reactant_smile_lst = row['reactant_smile'].split('.')
    if any(compare_smiles_inchikey(true_met,met) for met in reactant_smile_lst):
        print(row['ID'],row['reactant_smile'])

result_df

rxn3 O=C([O-])c1ccccc1.OO
rxn91 O.O=C([O-])c1ccccc1
rxn2 OCc1ccccc1
rxn92 OCc1ccccc1.OO


,ID,reactant_smile,productsmile,productsmile_num
1,rxn2,OCc1ccccc1,O=Cc1ccccc1,1
2,rxn3,O=C([O-])c1ccccc1.OO,O=Cc1ccccc1.O=O.[H]O[H],1
6,rxn8,O=Cc1c[C@@H](O)[C@@H](O)C=C1,O=Cc1ccccc1.O=O,1
7,rxn9,O=Cc1cC=C[C@H](O)[C@@H]1O,O=Cc1ccccc1.O=O,1
8,rxn10,O.O=Cc1ccc(O)cc1,O=Cc1ccccc1.O=O,1
9,rxn11,O.O=Cc1ccccc1O,O=Cc1ccccc1.O=O,1
10,rxn12,O.O=Cc1cccc(O)c1,O=Cc1ccccc1.O=O,1
13,rxn29,O.O=Cc1cC=C[C@@H]2O[C@H]12,O=Cc1ccccc1.O=O,1
14,rxn30,O.O=Cc1c[C@H]2O[C@H]2C=C1,O=Cc1ccccc1.O=O,1
33,rxn62,O.O=O.[NH3+]Cc1ccccc1,O=Cc1ccccc1.[NH4+].OO,1
